In [9]:
push!(LOAD_PATH,".. /src/")
using AgentBasedModels
using Makie
using Distributions
using DifferentialEquations

In [10]:
fσ(x) = (1+tanh(2*x))/2

fs0(x,t) = S0*fσ(1-t/τg)*(exp(-x^2/(2*L^2))+exp(-(1-x)^2/(2*L^2))) + fσ(t/τg-1)*(exp(-x^2/(2*l^2))+exp(-(1-x)^2/(2*l^2)))

fs0 (generic function with 1 method)

In [11]:
model = Model()

addGlobal!(model,[:a0,:a1,:τ,:l,:D,:S0,:τg,:L])

update =
"
s0 = fs0(x,t)
"
addLocal!(model,[:x,:y,:s0],updates=update)

coupling =
"
dₐ = minimum(
        [(x₁-x₂)^2+(y₁-y₂)^2,
        (x₁-x₂+1)^2+(y₁-y₂)^2,
        (x₁-x₂-1)^2+(y₁-y₂)^2,
        (x₁-x₂)^2+(y₁-y₂+1)^2,
        (x₁-x₂)^2+(y₁-y₂-1)^2,
        (x₁-x₂+1)^2+(y₁-y₂+1)^2,
        (x₁-x₂+1)^2+(y₁-y₂-1)^2,
        (x₁-x₂-1)^2+(y₁-y₂+1)^2,
        (x₁-x₂-1)^2+(y₁-y₂-1)^2])
if dₐ > 0
    s₁ += exp(-dₐ/(2*l^2)) * u₂*(a0 + 3*u₂^3*a1/(1+u₂^2))
end
"
addInteraction!(model,[:s],coupling)

eqs = 
"
du = (fσ(2*(u-s-s0))/τ -u/τ)*dt #+D*ξ_
"
addVariable!(model,[:u],eqs)

In [16]:
Lx = 1
Ly = 1
Nx = 18
Ny = 9

compile!(model,integrator="euler",saveRAM=true,platform="cpu")
com = Community(model,N=2*Nx*Ny)

#Global parameters
λ = 5*10^-6; a0 = .05; a1 = 1 - a0; τ = 1/2; l = 0.085#1.75*λ; 
D = 5*10^-5; S0 = 2; τg = 1; L = .2; N = 324
com[:a0] = a0
com[:a1] = a1
com[:τ] = τ
com[:l] = l
com[:D] = D
com[:s0] = S0
com[:L] = L

#Positions
dist = Uniform(-1,1)
posx = zeros(2*Nx*Ny); posy = zeros(2*Nx*Ny)
for i in 1:Nx
    for j in 1:Ny
        posx[Nx*(j-1)+i] = Lx*(i-0.5)/Nx +0.01*rand(dist)
        posy[Nx*(j-1)+i] = Ly*(j-0.5)/Ny +0.01*rand(dist)
    end
end
for i in 1:Nx
    for j in 1:Ny
        posx[Nx*Ny+Nx*(j-1)+i] = Lx*(i)/Nx +0.01*rand(dist)
        posy[Nx*Ny+Nx*(j-1)+i] = Ly*(j-0.5+cos(pi/3))/Ny +0.01*rand(dist)
    end
end

com[:x] = posx; com[:y] = posy;
#Concentration
u0 = [fs0(i,0.) for i in posx]
com[:u] = u0;
com[:s0] = u0;

LoadError: LoadError: UndefVarError: CUDA not defined
in expression starting at none:1

In [13]:
#figure = meshscatter(com[:x],com[:y],marker=Circle,markersize=5*10^-2,color=com[:u])
#display(figure)

In [15]:
c = model.evolve(com,dt=0.01,tMax_=10);

LoadError: UndefVarError: randn! not defined

In [338]:
comOut = c[1]
figure = meshscatter(comOut[:x],comOut[:y],marker=Circle,markersize=5*10^-2,color=comOut[:u])
#display(figure)
save("corson2017self1.png",figure)

In [339]:
comOut = c[250]
figure = meshscatter(comOut[:x],comOut[:y],marker=Circle,markersize=5*10^-2,color=comOut[:u])
#display(figure)
save("corson2017self250.png",figure)

In [342]:
comOut = c[500]
figure = meshscatter(comOut[:x],comOut[:y],marker=Circle,markersize=5*10^-2,color=comOut[:u])
#display(figure)
save("corson2017self500.png",figure)

In [343]:
comOut = c[750]
figure = meshscatter(comOut[:x],comOut[:y],marker=Circle,markersize=5*10^-2,color=comOut[:u])
#display(figure)
save("corson2017self750.png",figure)

In [344]:
comOut = c[1000]
figure = meshscatter(comOut[:x],comOut[:y],marker=Circle,markersize=5*10^-2,color=comOut[:u])
display(figure)
#display(figure)
save("corson2017self1000.png",figure)